In [513]:
import pandas as pd
import numpy as np
import copy
from datetime import datetime
import argparse
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score as auprc
from sklearn.metrics import roc_auc_score as auc_score
import keras
from keras.utils import multi_gpu_model
from keras.layers import Input, Dense, GRU, Lambda, Permute
from keras.models import Model
from interpolation_layer import single_channel_interp, cross_channel_interp
import warnings
from sklearn.preprocessing import KBinsDiscretizer
warnings.filterwarnings("ignore")
import random

In [514]:
admission = pd.read_csv('Dataset/temp_admission.csv')
admission = admission[["HADM_ID", "ADMISSION_TYPE", "LOS"]]
admission['LOS'] = admission['LOS'] * 24
admission = admission[admission['LOS'] >= hours_looks_ahead]

admission.head()

,HADM_ID,ADMISSION_TYPE,LOS
0,165315,EMERGENCY,27.466667
1,152223,ELECTIVE,131.916667
2,124321,EMERGENCY,162.433333
3,161859,EMERGENCY,68.566667
4,129635,EMERGENCY,84.816667


In [246]:

# test_ = item220045_.loc[item220045_['HADM_ID'] == 152223].sort_values(by=['CHARTTIME'])
# test1 = test_.groupby('HADM_ID')['CHARTTIME'].apply(list).tolist()[0]
# test2 = test_.groupby('HADM_ID')['VALUENUM'].apply(list)
# # for i in test1:
# #     print(i)
# # for j in test2:
# #     print(j)
# test1,test2

In [392]:
# Heart Rate
item220045 = pd.read_csv('Dataset/temp220045.csv')
item220045.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item220045['CHARTTIME'] = pd.to_datetime(item220045['CHARTTIME']).dt.tz_localize(None)
item220045 = item220045[item220045['VALUENUM'].notna()]

3548.7631926656572

In [371]:
# Arterial Blood Pressure systolic
item220050 = pd.read_csv('Dataset/temp220050.csv')
item220050.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'ITEM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item220050 = item220050[item220050['VALUENUM'].notna()]
item220050['CHARTTIME'] = pd.to_datetime(item220050['CHARTTIME']).dt.tz_localize(None)

In [372]:
# Non Invasive Blood Pressure systolic
item220179 = pd.read_csv('Dataset/temp220179.csv')
item220179.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item220179 = item220179[item220179['VALUENUM'].notna()]
item220179['CHARTTIME'] = pd.to_datetime(item220179['CHARTTIME']).dt.tz_localize(None)

In [373]:
# Temperature Fahrenheit
item223761 = pd.read_csv('Dataset/temp223761.csv')
item223761.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item223761 = item223761[item223761['VALUENUM'].notna()]
item223761['CHARTTIME'] = pd.to_datetime(item223761['CHARTTIME']).dt.tz_localize(None)

In [374]:
# Temperature C
item676 = pd.read_csv('Dataset/temp676.csv')
item676['CHARTTIME'] = pd.to_datetime(item676['CHARTTIME']).dt.tz_localize(None)
item676 = item676[item676['VALUENUM'].notna()]

In [375]:
# Inspired O2 Fraction (FiO2)
item223835 = pd.read_csv('Dataset/temp223835.csv')
item223835.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'ITEM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item223835 = item223835[item223835['VALUENUM'].notna()]
item223835['CHARTTIME'] = pd.to_datetime(item223835['CHARTTIME']).dt.tz_localize(None)

In [376]:
# RR
item618 = pd.read_csv('Dataset/temp618.csv')
item618['CHARTTIME'] = pd.to_datetime(item618['CHARTTIME']).dt.tz_localize(None)
item618 = item618[item618['VALUENUM'].notna()]

In [377]:
# RR should be merged with 618
item220210 = pd.read_csv('Dataset/temp220210.csv')
item220210['CHARTTIME'] = pd.to_datetime(item220210['CHARTTIME']).dt.tz_localize(None)
item220210 = item220210[item220210['VALUENUM'].notna()]

In [378]:
item_rr = item618.append(item220210, ignore_index=True)

In [379]:
# spo2 
item646 = pd.read_csv('Dataset/temp646.csv')
item646['CHARTTIME'] = pd.to_datetime(item646['CHARTTIME']).dt.tz_localize(None)
item646 = item646[item646['VALUENUM'].notna()]

In [380]:
# spo2 should be merged with 646
item220277 = pd.read_csv('Dataset/temp220277.csv')
item220277['CHARTTIME'] = pd.to_datetime(item220277['CHARTTIME']).dt.tz_localize(None)
item220277 = item220277[item220277['VALUENUM'].notna()]

In [381]:
item_spo2 = item646.append(item220277, ignore_index=True)

In [382]:
# DBP
item8441 = pd.read_csv('Dataset/temp8441.csv')
item8441['CHARTTIME'] = pd.to_datetime(item8441['CHARTTIME']).dt.tz_localize(None)
item8441 = item8441[item8441['VALUENUM'].notna()]

In [542]:
# 50882 serum bicarbon- ate level
item50882 = pd.read_csv('Dataset_bench/temp50882.csv')
item50882.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item50882['CHARTTIME'] = pd.to_datetime(item50882['CHARTTIME']).dt.tz_localize(None)
item50882 = item50882[item50882['VALUENUM'].notna()]

In [545]:
# 50971 potassium level
item50971 = pd.read_csv('Dataset_bench/temp50971.csv')
item50971.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item50971['CHARTTIME'] = pd.to_datetime(item50971['CHARTTIME']).dt.tz_localize(None)
item50971 = item50971[item50971['VALUENUM'].notna()]

In [544]:
# 50885 bilirubin level
item50885 = pd.read_csv('Dataset_bench/temp50885.csv')
item50885.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item50885['CHARTTIME'] = pd.to_datetime(item50885['CHARTTIME']).dt.tz_localize(None)
item50885 = item50885[item50885['VALUENUM'].notna()]

In [546]:
# 50983 sodium level
item50983 = pd.read_csv('Dataset_bench/temp50983.csv')
item50983.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item50983['CHARTTIME'] = pd.to_datetime(item50983['CHARTTIME']).dt.tz_localize(None)
item50983 = item50983[item50983['VALUENUM'].notna()]

In [547]:
# 51300 white blood cells count
item51300 = pd.read_csv('Dataset_bench/temp51300.csv')
item51300.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item51300['CHARTTIME'] = pd.to_datetime(item51300['CHARTTIME']).dt.tz_localize(None)
item51300 = item51300[item51300['VALUENUM'].notna()]

In [548]:
# 220739 glasgow coma scale
item220739 = pd.read_csv('Dataset_bench/temp220739.csv')
item220739.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item220739['CHARTTIME'] = pd.to_datetime(item220739['CHARTTIME']).dt.tz_localize(None)
item220739 = item220739[item220739['VALUENUM'].notna()]

In [837]:
# item 50822
item50822 = pd.read_csv('Dataset_bench/temp50822.csv')
item50822.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item50822['CHARTTIME'] = pd.to_datetime(item50822['CHARTTIME']).dt.tz_localize(None)
item50822 = item50822[item50822['VALUENUM'].notna()]

In [550]:
# urine output 226559 , 226560, 226561, 226563, 226564, 226565, 226567, 226584

item226559 = pd.read_csv('Dataset_bench/temp226559.csv')
item226559.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item226559['CHARTTIME'] = pd.to_datetime(item226559['CHARTTIME']).dt.tz_localize(None)
item226559 = item226559[item226559['VALUENUM'].notna()]

item226560 = pd.read_csv('Dataset_bench/temp226560.csv')
item226560.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item226560['CHARTTIME'] = pd.to_datetime(item226560['CHARTTIME']).dt.tz_localize(None)
item226560 = item226560[item226560['VALUENUM'].notna()]

item226561 = pd.read_csv('Dataset_bench/temp226561.csv')
item226561.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item226561['CHARTTIME'] = pd.to_datetime(item226561['CHARTTIME']).dt.tz_localize(None)
item226561 = item226561[item226561['VALUENUM'].notna()]

item226563 = pd.read_csv('Dataset_bench/temp226563.csv')
item226563.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item226563['CHARTTIME'] = pd.to_datetime(item226563['CHARTTIME']).dt.tz_localize(None)
item226563 = item226563[item226563['VALUENUM'].notna()]

item226564 = pd.read_csv('Dataset_bench/temp226564.csv')
item226564.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item226564['CHARTTIME'] = pd.to_datetime(item226564['CHARTTIME']).dt.tz_localize(None)
item226564 = item226564[item226564['VALUENUM'].notna()]

item226565 = pd.read_csv('Dataset_bench/temp226565.csv')
item226565.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item226565['CHARTTIME'] = pd.to_datetime(item226565['CHARTTIME']).dt.tz_localize(None)
item226565 = item226565[item226565['VALUENUM'].notna()]

item226567 = pd.read_csv('Dataset_bench/temp226567.csv')
item226567.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item226567['CHARTTIME'] = pd.to_datetime(item226567['CHARTTIME']).dt.tz_localize(None)
item226567 = item226567[item226567['VALUENUM'].notna()]

item226584 = pd.read_csv('Dataset_bench/temp226584.csv')
item226584.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item226584['CHARTTIME'] = pd.to_datetime(item226584['CHARTTIME']).dt.tz_localize(None)
item226584 = item226584[item226584['VALUENUM'].notna()]

In [551]:
item_uo = item226559.append(item226560, ignore_index=True)
item_uo = item_uo.append(item226561, ignore_index=True)
item_uo = item_uo.append(item226563, ignore_index=True)
item_uo = item_uo.append(item226564, ignore_index=True)
item_uo = item_uo.append(item226565, ignore_index=True)
item_uo = item_uo.append(item226567, ignore_index=True)
item_uo = item_uo.append(item226584, ignore_index=True)

In [798]:
# O2 50816
item50816 = pd.read_csv('Dataset_bench/temp50816.csv')
item50816.columns = ['ROW_ID', 'CHARTTIME', 'HADM_ID', 'SUBJECT_ID', 'VALUENUM', 'VALUEUOM']
item50816['CHARTTIME'] = pd.to_datetime(item50816['CHARTTIME']).dt.tz_localize(None)
item50816 = item50816[item50816['VALUENUM'].notna()]

In [919]:
# 52 
item52 = pd.read_csv('Dataset_bench/temp52.csv')
item52['CHARTTIME'] = pd.to_datetime(item52['CHARTTIME']).dt.tz_localize(None)
item52 = item52[item52['VALUENUM'].notna()]

In [840]:
# merge 50822 and 50971 Potassium
item_po = item50822.append(item50971, ignore_index=True)

In [920]:
vitals_dict = {}
np.random.seed(10)
num_features = 1
max_length = 2000 # length of timestamp
gpu_num = 0
ref_points = 192
hid = 100
batch = 128
epoch = 20
num_ids = 200
hours_looks_ahead = 24
num_fold = 3
feature_set = 2
stds = []

In [921]:
# return two lists of hadm_id: list of timestamps (ascending order), list of corresponding vals of the item
def time_val_toLst(df, hadmin_id):
    temp = df.loc[df['HADM_ID'] == hadmin_id].sort_values(by=['CHARTTIME'])
    if not temp.empty: 
        times = temp.groupby('HADM_ID')['CHARTTIME'].apply(list).tolist()[0]
        vals = temp.groupby('HADM_ID')['VALUENUM'].apply(list).tolist()[0]
        return [times, vals]
    else:
        return [] # if id does not have this item, return empty list

In [922]:
def flatten(df, los=hours_looks_ahead):
    a = np.full((len(df), num_features, max_length), -100) # initlizae all as missing
    timestamps = []
    for i in range(len(df)):
        l = []
    
        # find all the unique observed timestamps
        for j in range(num_features):
            ts_ij = df[i][j]
            if ts_ij != []:
                for ts in df[i][j][0]:
                    if ts not in l:
                        l.append(ts)
        l.sort()
        T = copy.deepcopy(l)
        TS = []
        for t in T:
            if (t - T[0]).total_seconds() / 3600 <= los:
                TS.append(t)
        timestamps.append(TS)
        for j in range(num_features):
            s_ij = df[i][j]
            if s_ij != []:
                ts_ij = s_ij[0] 
                c_max = len(ts_ij)
                c = 0
                for k in range(len(TS)):
                    if c < c_max:
                        ts_ijc = ts_ij[c] # cur ts
                        diff = abs(TS[k] - ts_ijc).seconds / 60 # difference between ts's
                        if TS[k] == ts_ijc or diff < 5:
                            a[i,j,k] = s_ij[1][c] 
                            c += 1
    return a, timestamps

In [923]:
def input_format(x, T):
    real_len = 200
    
    for i in range(len(T)):
        if len(T[i]) > real_len:
            T[i] = T[i][:real_len]

    x = a[:, :, :real_len]
    M = np.zeros_like(x)
    delta = np.zeros_like(x)

    for t in T:
        for i in range(1, len(t)):
            t[i] = (t[i] - t[0]).total_seconds() / 3600 # hours difference
        if len(t) != 0:
            t[0] = 0
    
    # count outliers and negative values as missing values
    # M = 0 indicates missing value
    # M = 1 indicates observed value
    # now since we have mask variable, we don't need -100
    M[x > 500] = 0
    x[x > 500] = 0.0
    M[x < 0] = 0
    x[x < 0] = 0.0
    M[x > 0] = 1

    for i in range(num_features):
        for j in range(x.shape[0]):
            for k in range(len(T[j])):
                delta[j, i, k] = T[j][k]
    return x, M, delta

In [924]:
def missing_mean(M, x):
    counts = np.sum(np.sum(M, axis=2), axis=0)
    mean_values = np.sum(np.sum(x*M, axis=2), axis=0)/counts
    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            if np.sum(M[i, j]) == 0:
                M[i, j, 0] = 1
                x[i, j, 0] = mean_values[j]
    return 

In [925]:
def hold_out(mask, perc=0.2):
    """To implement the autoencoder component of the loss, we introduce a set
    of masking variables mr (and mr1) for each data point. If drop_mask = 0,
    then we removecthe data point as an input to the interpolation network,
    and includecthe predicted value at this time point when assessing
    the autoencoder loss. In practice, we randomly select 20% of the
    observed data points to hold out from
    every input time series."""
    drop_mask = np.ones_like(mask)
    drop_mask *= mask
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            count = np.sum(mask[i, j], dtype='int')
            if int(0.20*count) > 1:
                index = 0
                r = np.ones((count, 1))
                b = np.random.choice(count, int(0.20*count), replace=False)
                r[b] = 0
                for k in range(mask.shape[2]):
                    if mask[i, j, k] > 0:
                        drop_mask[i, j, k] = r[index]
                        index += 1
    return drop_mask

In [926]:
# random.seed(123)
adm_ids = admission['HADM_ID'].tolist()
needed_ids = random.sample(adm_ids,  num_ids)

In [927]:
# create the vitals dictionary: {id: [item1, item2, item3, item4, item5]}
# item1 = [list of timestamps, list of vals]
# only use the first 200 hadm_ids as sample

# 220045, spo2, rr,  
# needed_ids = adm_ids[:num_ids]
for adm_id in needed_ids:
    if feature_set == 1:
        vitals_dict[adm_id] = [time_val_toLst(item220045, adm_id)]
        vitals_dict[adm_id].append(time_val_toLst(item220050, adm_id))
        vitals_dict[adm_id].append(time_val_toLst(item220179, adm_id))
        vitals_dict[adm_id].append(time_val_toLst(item676, adm_id))
        vitals_dict[adm_id].append(time_val_toLst(item223835, adm_id))
        vitals_dict[adm_id].append(time_val_toLst(item_rr, adm_id))
        vitals_dict[adm_id].append(time_val_toLst(item_spo2, adm_id))
        vitals_dict[adm_id].append(time_val_toLst(item8441, adm_id))
        
    if feature_set == 2:
#         vitals_dict[adm_id] = [time_val_toLst(item676, adm_id)]
#         vitals_dict[adm_id] = [time_val_toLst(item50816, adm_id)]
#         vitals_dict[adm_id].append(time_val_toLst(item50882, adm_id))
#         vitals_dict[adm_id].append(time_val_toLst(item50885, adm_id))
        vitals_dict[adm_id] = [time_val_toLst(item52, adm_id)]
#         vitals_dict[adm_id].append(time_val_toLst(item50971, adm_id))
#         vitals_dict[adm_id].append(time_val_toLst(item50983, adm_id))
#         vitals_dict[adm_id].append(time_val_toLst(item51300, adm_id))
#         vitals_dict[adm_id].append(time_val_toLst(item220045, adm_id))
#         vitals_dict[adm_id].append(time_val_toLst(item220179, adm_id))
#         vitals_dict[adm_id].append(time_val_toLst(item220739, adm_id))
#         vitals_dict[adm_id].append(time_val_toLst(item223835, adm_id))
#         vitals_dict[adm_id].append(time_val_toLst(item_uo, adm_id))
#         vitals_dict[adm_id] = [time_val_toLst(item_po, adm_id)]
            


In [928]:
stds = []
if feature_set == 1:
    stds.append(item220045['VALUENUM'].std())
    stds.append(item220050['VALUENUM'].std())
    stds.append(item220179['VALUENUM'].std())
    stds.append(item676['VALUENUM'].std())
    stds.append(item223835['VALUENUM'].std())
    stds.append(item_rr['VALUENUM'].std())
    stds.append(item_spo2['VALUENUM'].std())
    stds.append(item8441['VALUENUM'].std())
if feature_set == 2:
#     stds.append(item50816['VALUENUM'].std())
#     stds.append(item676['VALUENUM'].std())
#     stds.append(item50822['VALUENUM'].std())
#     stds.append(item50882['VALUENUM'].std())
#     stds.append(item50885['VALUENUM'].std())
#     stds.append(item50971['VALUENUM'].std())
#     stds.append(item50983['VALUENUM'].std())
#     stds.append(item51300['VALUENUM'].std())
#     stds.append(item220045['VALUENUM'].std())
#     stds.append(item220179['VALUENUM'].std())    
#     stds.append(item220739['VALUENUM'].std())
#     stds.append(item223835['VALUENUM'].std())
#     stds.append(item_uo['VALUENUM'].std()) 
#     stds.append(item_po['VALUENUM'].std()) 
    stds.append(item52['VALUENUM'].std()) 

In [929]:
# def y_trans(val):
#     if val <= 48:
#         return 0
#     elif 48 < val <= 168:
#         return 1
#     else:
#         return 2

In [930]:
vitals = [vitals_dict[x] for x in needed_ids] # hadm_id(los>=48h): all the vitals values
kb = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='kmeans')
label = kb.fit_transform(admission[admission['HADM_ID'].isin(needed_ids)].LOS.to_numpy().reshape(-1,1))
# label = admission[admission['HADM_ID'].isin(needed_ids)].LOS.tolist()
# label = [y_trans(l) for l in label]
print(kb.bin_edges_ / 24)

[array([  1.04930556,  32.30618989,  89.80744949, 122.91527778])]


In [931]:
a, ts = flatten(vitals, hours_looks_ahead)
x, m, T = input_format(a, ts)
missing_mean(m, x)
X = np.concatenate((x, m, T, hold_out(m)), axis=1)  # input format
y = np.array(label)
print(X.shape, y.shape)
timestamp = X.shape[2]
num_features = X.shape[1] // 4

(200, 4, 200) (200, 1)


In [932]:
num_features

1

In [933]:
# [17.3, 22, 22.45, 2.32, 23.41，5.7, 3.33, 14.1]
def customloss(ytrue, ypred):
    """ Autoencoder loss
    """
    num_fs = num_features
    # standard deviation of each feature mentioned in paper for MIMIC_III data
    wc = np.array(stds)
    print(wc.shape, ytrue.shape)
    wc.shape = (1, num_fs)
    y = ytrue[:, :num_fs, :]
    m2 = ytrue[:, 3*num_fs:4*num_fs, :]
    m2 = 1 - m2
    m1 = ytrue[:, num_fs:2*num_fs, :]
    m = m1*m2
    ypred = ypred[:, :num_fs, :]
    x = (y - ypred)*(y - ypred)
    x = x*m
    count = tf.reduce_sum(m, axis=2)
    count = tf.where(count > 0, count, tf.ones_like(count))
    x = tf.reduce_sum(x, axis=2)/count
    x = x/(wc**2)  # dividing by standard deviation
    x = tf.reduce_sum(x, axis=1)/num_fs
    return tf.reduce_mean(x)

In [934]:
def interp_net():
    num_fs = num_features
    if gpu_num > 1:
        dev = "/cpu:0"
    else:
        dev = "/gpu:0"
    with tf.device(dev):
        main_input = Input(shape=(4*num_fs, timestamp), name='input')
        sci = single_channel_interp(ref_points, hours_looks_ahead)
        cci = cross_channel_interp()
        interp = cci(sci(main_input))
        reconst = cci(sci(main_input, reconstruction=True),
                      reconstruction=True)
        aux_output = Lambda(lambda x: x, name='aux_output')(reconst)
        z = Permute((2, 1))(interp)
        z = GRU(hid, activation='tanh', recurrent_dropout=0.2, dropout=0.2)(z)
        main_output = Dense(1, activation='sigmoid', name='main_output')(z)
        orig_model = Model([main_input], [main_output, aux_output])
    if gpu_num > 1:
        model = multi_gpu_model(orig_model, gpus=gpu_num)
    else:
        model = orig_model
    print(orig_model.summary())
    return model

In [935]:
seed = 0
results = {}
results['loss'] = []
results['auc'] = []
results['acc'] = []
results['auprc'] = []
earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0000, patience=20, verbose=0)
callbacks_list = [earlystop]


In [936]:
# 5-fold cross-validation
i = 0
kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
for train, test in kfold.split(np.zeros(len(y)), y):
    print("Running Fold:", i+1)
    model = interp_net()  # re-initializing every time
    model.compile(
        optimizer='adam',
        loss={'main_output': 'binary_crossentropy', 'aux_output': customloss},
        loss_weights={'main_output': 0.99, 'aux_output': 0.99},
        metrics={'main_output': 'accuracy'})
    model.fit(
        {'input': X[train]}, {'main_output': y[train], 'aux_output': X[train]},
        batch_size=batch,
        callbacks=callbacks_list,
        nb_epoch=epoch,
        validation_split=0.20,
        verbose=2)
    y_pred = model.predict(X[test], batch_size=batch)
    y_pred = y_pred[0]
    total_loss, score, reconst_loss, acc = model.evaluate(
        {'input': X[test]},
        {'main_output': y[test], 'aux_output': X[test]},
        batch_size=batch,
        verbose=0)
    results['loss'].append(score)
    results['acc'].append(acc)
#     results['auc'].append(auc_score(y[test], y_pred))
    results['auprc'].append(auprc(y[test], y_pred))
    print(results)
    i += 1

Running Fold: 1
Model: "model_93"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 4, 200)       0                                            
__________________________________________________________________________________________________
single_channel_interp_93 (singl multiple             1           input[0][0]                      
                                                                 input[0][0]                      
__________________________________________________________________________________________________
cross_channel_interp_93 (cross_ multiple             1           single_channel_interp_93[0][0]   
                                                                 single_channel_interp_93[1][0]   
___________________________________________________________________________

(1,) (?, ?, ?)
Train on 80 samples, validate on 20 samples
Epoch 1/20
 - 17s - loss: 6532043961794560.0000 - main_output_loss: 6598024323137536.0000 - aux_output_loss: 0.0591 - main_output_accuracy: 0.8500 - val_loss: 0.3372 - val_main_output_loss: 0.3018 - val_aux_output_loss: 0.0388 - val_main_output_accuracy: 1.0000
Epoch 2/20
 - 1s - loss: 7214964699824128.0000 - main_output_loss: 7287843315515392.0000 - aux_output_loss: 0.0592 - main_output_accuracy: 0.7375 - val_loss: 0.3295 - val_main_output_loss: 0.2940 - val_aux_output_loss: 0.0388 - val_main_output_accuracy: 1.0000
Epoch 3/20
 - 1s - loss: 9702528690159616.0000 - main_output_loss: 9800533401403392.0000 - aux_output_loss: 0.0592 - main_output_accuracy: 0.8750 - val_loss: 0.3242 - val_main_output_loss: 0.2887 - val_aux_output_loss: 0.0388 - val_main_output_accuracy: 1.0000
Epoch 4/20
 - 1s - loss: 3752466755616768.0000 - main_output_loss: 3790370378874880.0000 - aux_output_loss: 0.0593 - main_output_accuracy: 0.8375 - val_loss:

ValueError: multiclass format is not supported

In [937]:
print(results)

{'loss': [0.4442867338657379, nan], 'auc': [], 'acc': [0.949999988079071, 0.0], 'auprc': [0.2409560723514212]}


In [665]:
{'loss': [0.4442867338657379, nan], 'auc': [], 'acc': [0.949999988079071, 0.0], 'auprc': [0.2409560723514212]}

{'loss': [0.8490171432495117,
  0.8478379249572754,
  0.5991557240486145,
  0.20699283480644226,
  1.7641651630401611],
 'auc': [0.44594594594594594,
  0.4864864864864865,
  0.5263157894736843,
  0.5263157894736842,
  0.5789473684210527],
 'acc': [0.07500000298023224,
  0.07500000298023224,
  0.949999988079071,
  0.949999988079071,
  0.05000000074505806],
 'auprc': [0.075,
  0.07894736842105263,
  0.05263157894736842,
  0.05555555555555555,
  0.058823529411764705]}

In [296]:
# {'loss': [0.7575289607048035, 0.30154579877853394, 0.14339998364448547, nan], 'auc': [], 'acc': [0.42500001192092896, 0.9750000238418579, 0.9750000238418579, 0.0], 'auprc': [0.06666666666666667, 0.3333333333333333, 0.09090909090909091]}
# for lst in timestamps:
#     if len(lst) > max_length:
#         timestamps[i] = lst[:max_length]

# x = a[:, :, :max_length]
# M = np.zeros_like(x)
# delta = np.zeros_like(x)

# for t in timestamps:
#     for i in range(1, len(t)):
#         t[i] = (t[i] - t[0]).total_seconds() / 3600 # hours difference
#     if len(t) != 0:
#         t[0] = 0

In [220]:
# a = np.full((len(vitals), num_features, max_length), -100) # initlizae all as missing
# timestamps = []
# for i in range(len(vitals)):
#     l = []
    
#     # find all the unique observed timestamps
#     for j in range(num_features):
#         ts_ij = vitals[i][j]
#         if ts_ij != []:
#             for ts in vitals[i][j][0]:
#                 if ts not in l:
#                     l.append(ts)
#     l.sort()
#     T = copy.deepcopy(l)
#     TS = []
#     for t in T:
#         if (t - T[0]).total_seconds() / 3600 <= 72:
#             TS.append(t)
#     timestamps.append(TS)
#     for j in range(num_features):
#         s_ij = vitals[i][j]
#         if s_ij != []:
#             ts_ij = s_ij[0] 
#             c_max = len(ts_ij)
#             c = 0
#             for k in range(len(TS)):
#                 if c < c_max:
#                     ts_ijc = ts_ij[c] # cur ts
#                     diff = abs(TS[k] - ts_ijc).seconds / 60 # difference between ts's
#                     if TS[k] == ts_ijc or diff < 5:
#                         a[i,j,k] = s_ij[1][c] 
#                         c += 1

In [238]:
# # count outliers and negative values as missing values
# # M = 0 indicates missing value
# # M = 1 indicates observed value
# # now since we have mask variable, we don't need -100
# M[x > 500] = 0
# x[x > 500] = 0.0
# M[x < 0] = 0
# x[x < 0] = 0.0
# M[x > 0] = 1

# for i in range(num_features):
#         for j in range(x.shape[0]):
#             for k in range(len(timestamps[j])):
#                 delta[j, i, k] = timestamps[j][k]

In [185]:
x = np.concatenate((x, m, T, hold_out(m)), axis=1)  # input format
y = np.array(label)
print(x.shape, y.shape)
timestamp = x.shape[2]
num_features = x.shape[1] // 4

[39, 38, 13, 33, 22, 2, 37, 21, 29, 32, 27, 31, 53, 35, 47, 30, 31, 44, 25, 28]

In [429]:
results

{'loss': [0.18130822479724884, nan],
 'auc': [0.6025641025641025],
 'acc': [0.9750000238418579, 0.0],
 'auprc': [0.04805491990846682]}